In [120]:
%matplotlib widget

# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False 

# TITLE
print("Create your own DotSpace!")

Create your own DotSpace!


In [121]:
# INFO
print("Size: Grid size of square")
print("Pattern:"+"\n"+" [example 1] 6 will display one black square followed by five white squares "+"\n"+" [example 2] 4 will display one black square followed by three white squares "+"\n"+" [example 3] 3,4 will display one black square, two white squares, one black square, three white squares ")
print("Layered: Single layer or two layers at right angles on top of one another")

Size: Grid size of square
Pattern:
 [example 1] 6 will display one black square followed by five white squares 
 [example 2] 4 will display one black square followed by three white squares 
 [example 3] 3,4 will display one black square, two white squares, one black square, three white squares 
Layered: Single layer or two layers at right angles on top of one another


In [129]:
# WIDGETS

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display,clear_output



# Create the widgets and display the top container.
size = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Size',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='d')
pattern = widgets.BoundedIntText(
    value=4,
    min=0,
    max=9999999999,
    step=1,
    description='Pattern',
    disabled=False
)
layered = widgets.RadioButtons(
    options=['single layer', 'double layer'],
    value='single layer',
    description='Layers:',
    disabled=False
)


out1 = widgets.Output()
button = widgets.Button(description="Generate DotSpace")
hbox = widgets.HBox(children=[size,pattern,layered])
vbox = widgets.VBox(children=[hbox, button, out1])
display(vbox)

plt.ioff()
ax=plt.gca()

def on_button_clicked(b):
    generate_dotspace(size.value,pattern.value,layered.value)
    with out1:
        clear_output(wait=True)
        display(ax.figure)
        
# DOTSPACE BACKEND

def create(args, orientation):

    pattern = np.zeros(args["width"]*args["height"], dtype=int)
    
    if orientation == 90:
        steps = 1
        basis = args["digital"]
    elif orientation == 0:
        steps = 0
        basis = args["digital"]

    basis_length = len(basis)
    

    # Working in 1D
    counter = 0 
    for position in np.arange(len(pattern)):
        if basis[counter] == 1:
            pattern[position] = 1
        counter = ((counter + 1) % basis_length) # clock maths!
    
    # 2D to the eye
    pattern.shape = (args["height"], args["width"])
    pattern = np.rot90(pattern,4-steps)    
    return pattern

def overlay(pattern):

    length = pattern[0].shape[0] # We know they are square
    pattern_overlaid = np.zeros((length, length))
    
    for i in np.arange(length):
        for j in np.arange(length):
            if any(x[i,j] == 1 for x in pattern):
               pattern_overlaid[i,j] = 1
               
    return pattern_overlaid

def visualise(args, pattern):
    
    plt.set_cmap('Greys')
    ax.imshow(pattern, interpolation='nearest') 
    plt.axis('off')


def generate_dotspace(size,pattern,layered):
    
    ax.clear()

    args = dict()        
    args["dpi"] = 96
    args["basis"] = pattern
    args["digital"] = []
    digits_list = list(map(int,str(pattern)))
    for x in digits_list:
        args["digital"] = args["digital"] + [1] + [0]*(x-1)
    args["width"] = size
    args["height"] = size
    
    patterns = []
    patterns.append(create(args,0))
    
    if layered == "double layer":
        patterns.append(create(args,90))
        patterns.append(overlay(patterns))

    pattern = patterns[-1]
    visualise(args, pattern)


button.on_click(on_button_clicked)
on_button_clicked(None)